In [1]:
import config as cfg

In [2]:
import pandas as pd
from path_utils import get_file_path

In [ ]:
RAW_SYM_DIR = "./raw_data/SYM"
# 엑셀 파일 경로 (실제 경로로 수정)
output_folder_name = "./tmp/SYM"
SYM1_file_name = "backup_SYM1.xlsx"
SYM2_file_name = "backup_SYM2.xlsx"

SYM_raw_paths = [
    get_file_path(RAW_SYM_DIR, f"{SYM1_file_name}"),
    get_file_path(RAW_SYM_DIR, f"{SYM2_file_name}"),
]
output_folder = get_file_path(output_folder_name)

In [2]:
import pandas as pd
import numpy as np
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids
from functools import reduce
import datetime
import os 

start_date = load_raw_file(SYM_raw_paths, sheet_name="연구 참여자 기본 정보")

start_date = start_date.rename(columns={'비식별키': 'ID', '연구_동의일': 'start_date'})
start_date = start_date[['ID', 'start_date']]
start_date['start_date'] = pd.to_datetime(start_date['start_date'], errors='coerce').dt.date
output_path = os.path.join(output_folder, "start_date.csv")
start_date.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids
import datetime
import numpy as np
# 1. 엑셀 파일 경로 리스트

# 2. “공황일지” 시트를 모두 불러와 합치기
sdm = load_raw_file(SYM_raw_paths, sheet_name='생활패턴-흡연,식사,생리')




# ——— 2. 이진 변수 변환 ———
# 생리 여부 Y → 1, else 0
sdm['생리'] = np.where(sdm['생리'] == 'Y', 1, np.nan)
# 야식 여부 Y → 1, else 0
sdm['야식'] = np.where(sdm['야식'] == 'Y', 1, 0)

# ——— 3. 불필요 컬럼 제거 ———
sdm.drop(
    columns=['아침식사','점심식사','저녁식사','오전간식','오후간식'],
    inplace=True
)

# ——— 4. 컬럼명 정리 ———
sdm = sdm.rename(columns={
    '비식별키': 'ID',
    '날짜': 'date',
    '흡연량': 'smoking',
    '야식': 'late_night_snack',
    '생리': 'menstruation'
})[['ID','date','smoking','late_night_snack','menstruation']].copy()
sdm = filter_by_valid_ids(sdm, id_column='ID')
# ——— 5. 인덱스 리셋 & 결과 저장 ———
sdm.reset_index(drop=True, inplace=True)
output_path = os.path.join(output_folder, "smoking_diet_mens.csv")
sdm.to_csv(output_path, index=False)

In [5]:
import pandas as pd
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids

# 2. “생활패턴-음주” 시트를 모두 불러와 합치기
sleep_raw = load_raw_file(SYM_raw_paths, sheet_name="라이프로그-수면")

sleep_raw = filter_by_valid_ids(sleep_raw, id_column="비식별키")


# 4) 날짜 컬럼 datetime으로 변환하고 normalize
sleep_raw['날짜'] = pd.to_datetime(sleep_raw['날짜'], errors='coerce').dt.normalize()

# 5) 측정값 컬럼명 자동 탐색
measure_cols = [col for col in sleep_raw.columns if '측정값' in col]
if not measure_cols:
    raise KeyError("측정값 컬럼을 찾을 수 없습니다. 컬럼명을 확인하세요.")
# 여러 개 있으면 첫 번째를 사용
measure_col = measure_cols[0]

print("사용할 측정값 컬럼:", measure_col)

# 5.5) 동일한 ID, 날짜에 대해 측정값 이어붙이기
sleep_raw = sleep_raw.groupby(['비식별키', '날짜'], as_index=False).agg({
    measure_col: lambda x: ','.join(x.dropna().astype(str)),
    '취침시간': 'first',
    '기상시간': 'first'
})

# 6) SLT별 누적 시간을 계산하는 함수
def calc_slt_times(row):
    raw = row[measure_col]
    # 쉼표로 구분된 문자열 → 정수 리스트
    vals = [int(x) for x in str(raw).split(',') if x != '']
    # 개수 세기
    count_0 = vals.count(0)  # SLT2
    count_1 = vals.count(1)  # SLT1
    count_2 = vals.count(2)  # SLT6
    count_3 = vals.count(3)  # SLT4
    count_4 = vals.count(4)  # SLT5
    # SLT3은 해당값 없음
    count_5 = 0

    # 30초 단위 → 시간(시간 단위)
    slt1_h = count_1 * 30 / 3600
    slt2_h = count_0 * 30 / 3600
    slt3_h = 0
    slt4_h = count_3 * 30 / 3600
    slt5_h = count_4 * 30 / 3600
    slt6_h = count_2 * 30 / 3600

    return pd.Series({
        'SLT1': slt1_h,
        'SLT2': slt2_h,
        'SLT3': slt3_h,
        'SLT4': slt4_h,
        'SLT5': slt5_h,
        'SLT6': slt6_h
    })

# 7) calc_slt_times를 df_filtered에 적용
slt_times = sleep_raw.apply(calc_slt_times, axis=1)

# 8) 필요한 컬럼(ID, 날짜, 취침시간, 기상시간)과 SLT 결과 합치기
df_final = pd.concat([
    sleep_raw[['비식별키', '날짜', '취침시간', '기상시간']]
        .rename(columns={'비식별키': 'ID', '날짜': 'date'})
        .reset_index(drop=True),
    slt_times.reset_index(drop=True)
], axis=1)

# 9) 'ID'·'date' 순으로 정렬
df_final = df_final.sort_values(['ID', 'date']).reset_index(drop=True)

# Convert 취침시간 and 기상시간 to datetime and compute total_sleep
df_final['취침시간'] = pd.to_datetime(df_final['취침시간'], errors='coerce')
df_final['기상시간'] = pd.to_datetime(df_final['기상시간'], errors='coerce')
df_final['total_sleep'] = (df_final['기상시간'] - df_final['취침시간']).dt.total_seconds() / 3600
# Drop 취침시간 and 기상시간 columns
df_final.drop(columns=['취침시간', '기상시간'], inplace=True)

# 10) 결과 확인 (상위 10행 출력)
print(df_final.head(10))

output_path = os.path.join(output_folder, "sleep_summary.csv")
df_final.to_csv(output_path, index=False)

사용할 측정값 컬럼: 측정값(_1_:_값_없음,_0_:_알수_없는_수면,_1_:_깨어남,_2_:_램_수면,_3_:_얕은_수면,_4_:_깊은_수면)
           ID       date  SLT1      SLT2  SLT3      SLT4      SLT5      SLT6  \
0  SYM1-1-100 2021-03-19   0.0  8.000000   0.0  0.000000  0.000000  0.000000   
1  SYM1-1-100 2021-08-12   0.9  0.000000   0.0  3.683333  1.225000  1.516667   
2  SYM1-1-103 2021-02-17   0.0  0.000000   0.0  5.333333  1.833333  0.000000   
3  SYM1-1-103 2021-02-18   0.0  4.250000   0.0  2.450000  0.566667  0.000000   
4  SYM1-1-103 2021-02-19   0.0  0.000000   0.0  6.166667  1.566667  0.000000   
5  SYM1-1-103 2021-02-23   0.0  0.033333   0.0  0.000000  0.000000  0.000000   
6  SYM1-1-103 2021-03-04   0.0  7.000000   0.0  0.000000  0.000000  0.000000   
7  SYM1-1-103 2021-03-06   0.0  0.000000   0.0  0.033333  0.000000  0.000000   
8  SYM1-1-103 2021-03-08   0.0  4.766667   0.0  1.816667  0.000000  0.000000   
9  SYM1-1-103 2021-04-20   0.0  0.000000   0.0  3.050000  0.000000  0.000000   

   total_sleep  
0     8.000000  
1  

In [9]:
import pandas as pd
import numpy as np
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids
from functools import reduce

def extract_questionnaire_from_raw(path, questionnaire_sheet, df_name, questionnaire_column):
    df = load_raw_file(path, sheet_name=questionnaire_sheet)
    # Load 비식별키, 설문시작일, 설문완료일, and questionnaire_column
    df = df[['비식별키', '설문시작일', '설문완료일', questionnaire_column]]
    # Filter to keep only rows where 설문완료일 is not null
    df = df[pd.notnull(df['설문완료일'])].copy()
    # Use 설문시작일 as date
    df = df[['비식별키', '설문시작일', questionnaire_column, '설문완료일']]
    # Drop the 설문완료일 column
    df.drop(['설문완료일'], axis=1, inplace=True)
    # Rename columns to ["ID", "date", df_name]
    df.columns = ["ID", "date", df_name]
    df.drop_duplicates(['ID','date'], keep='last', inplace=True, ignore_index=False)
    return df


# ——— 1. 각 설문지별 시트 이름과 문항 컬럼 매핑 ———
questionnaire_specs = {
    'BRIAN':  ('22생물학적 리듬',    '1.생물학적_리듬_평가_척도_1번_히든_그룹_합산'),
    'CSM':    ('20아침형-저녁형', '1.조합_척도_1~13번_문항_점수_합산'),
    'CTQ_1':  ('27유년기 외상',       '1.유년기_외상_척도_요인1._정서방임_점수_합산'),
    'CTQ_2':  ('27유년기 외상',       '2.유년기_외상_척도_요인2._신체학대_점수_합산'),
    'CTQ_3':  ('27유년기 외상',       '3.유년기_외상_척도_요인3._성학대_점수_합산'),
    'CTQ_4':  ('27유년기 외상',       '4.유년기_외상_척도_요인4._정서학대_점수_합산'),
    'CTQ_5':  ('27유년기 외상',       '5.유년기_외상_척도_요인5._신체방임_점수_합산'),
    'KRQ':    ('13회복탄력성',              '1.회복탄력성_척도_1번_그룹_점수_합산'),
    'MDQ':    ('2기분 장애',          '1.기분_장애_척도_1번_그룹_점수_합산'),
    'SPAQ_1': ('21계절성 양상',             '1.계절성_양상_척도_2번_그룹_점수_합산'),
    'SPAQ_2': ('21계절성 양상',             '2.계절성_양상_척도_3번_문항_점수'),
    'STAI_X2':('5특성 불안',           '1.특성_불안_척도_1번_그룹_점수_합산'),
    'ACQ':    ('9광장공포인지', '1.광장공포_인지_척도_1번_그룹_점수_합산'),
    'APPQ_1': ('11공황-공포',         '1.알바니_공포_공황_척도_요인1._광장공포_점수_합계'),
    'APPQ_2': ('11공황-공포',         '2.알바니_공포_공황_척도_요인2._사회공포_점수_합계'),
    'APPQ_3': ('11공황-공포',         '3.알바니_공포_공황_척도_요인3._내부감각두려움_점수_합계'),
    'BSQ':    ('10신체감각','1.신체감각_척도_1번_그룹_점수_합산'),
    'BFNE':   ('6부정적평가에 대한 두려움','1.두려움_척도_1번_그룹_점수_합산'),
    'CES_D':  ('32우울증',         '1.우울_척도_개정판_1번_그룹_점수_합산'),
    'GAD_7':  ('8범불안 장애','1.범불안_장애_척도_1번_그룹_점수_합산'),
    'KOSSSF': ('12직무스트레스',         '1.직무스트레스_단축형_척도_1번_그룹_점수_합산'),
    'PHQ_9':  ('1우울증 선별','1.우울증_척도_1번_그룹_점수_합산'),
    'SADS':   ('7사회적회피 및 불편감','1.사회적_회피_및_불편감_척도_1번_그룹_점수_합산'),
    'STAI_X1':('4상태 불안',       '1.불안_척도_1번_그룹_점수_합산'),
}



# ——— 2. SYM1·SYM2 파일에서 같은 설문지만 모아오는 함수 ———
def extract_multi(path, sheet_name, df_name, questionnaire_col):
    dfs = []
    for p in path:
        df = extract_questionnaire_from_raw(
            path=p,
            questionnaire_sheet=sheet_name,
            df_name=df_name,
            questionnaire_column=questionnaire_col
        )
        dfs.append(df)
    # 동일 ID·date 중복 시 마지막 값을 남기고 제거
    return pd.concat(dfs, ignore_index=True) \
             .drop_duplicates(subset=['ID','date'], keep='last')

# ——— 3. 각 설문지 데이터 로드 ———
loaded = {}
for name, (sheet, col) in questionnaire_specs.items():
    loaded[name] = extract_multi(SYM_raw_paths, sheet, name, col)

# ——— 4. 날짜 컬럼 datetime 변환 ———
for df in loaded.values():
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

# ——— 5. 여러 설문지 병합 (outer join) ———
questionnaire_bydate = reduce(
    lambda left, right: pd.merge(left, right, on=['ID','date'], how='outer'),
    loaded.values()
)

# ——— 6. 수치형 변환 ———
for col in loaded.keys():
    questionnaire_bydate[col] = pd.to_numeric(questionnaire_bydate[col], errors='coerce')

# ——— 7. 결과 저장 (CSV) ———
questionnaire_bydate = filter_by_valid_ids(questionnaire_bydate, id_column="ID")
output_path = os.path.join(output_folder, "questionnaire.csv")
questionnaire_bydate.to_csv(output_path, index=False)

In [10]:
import pandas as pd
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids

# 2. “공황일지” 시트를 모두 불러와 합치기
panic_raw = load_raw_file(SYM_raw_paths, sheet_name="공황일지")

# 3. 필요한 컬럼을 동적으로 찾기
cols = panic_raw.columns.tolist()

# - ID: “비식별키” 포함
id_col = next((c for c in cols if "비식별키" in c), None)
# - date: “날짜” 포함
date_col = next((c for c in cols if "날짜" in c or "Date" in c), None)
# - severity(강도): “강도” 또는 “Severity” 포함
severity_col = next((c for c in cols if "강도" in c or "Severity" in c), None)

# 4. 컬럼 존재 여부 확인
missing = [name for name, col in [("ID", id_col), ("date", date_col), ("severity", severity_col)]
           if col is None]
if missing:
    raise KeyError(f"다음 필수 컬럼을 찾을 수 없습니다: {missing}\n실제 컬럼명: {cols}")

# 5. ID, date, severity 컬럼만 추출

panic = panic_raw[[id_col, date_col, severity_col]].copy()

# 6. 날짜를 YYYY-MM-DD 문자열로 변환
panic['date'] = pd.to_datetime(panic[date_col], errors='coerce').dt.strftime("%Y-%m-%d")

# 7. panic = 2로 일괄 설정 (데이터가 존재하면 공황 기록이 있다는 의미)
panic['panic'] = 2

# 8. 컬럼 이름 통일
panic.rename(columns={
    id_col: 'ID',
    severity_col: 'severity'
}, inplace=True)

# 9. 이제 필요한 최종 컬럼 순서만 남기기
panic = panic[['ID', 'date', 'panic', 'severity']]
# 10. 유효한 ID만 남기기
panic = filter_by_valid_ids(panic, id_column="ID")
# 11. 같은 ID, date에 중복된 행이 있을 경우 severity와 panic의 최대값만 남기기
panic = panic.groupby(['ID', 'date'], as_index=False).agg({
    'panic': 'max',
    'severity': 'max'
})

output_path = os.path.join(output_folder, "panic_by_date.csv")
panic.to_csv(output_path, index=False)

In [11]:
import pandas as pd
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids

# 2. “생활패턴-음주” 시트를 모두 불러와 합치기
alcohol_raw = load_raw_file(SYM_raw_paths, sheet_name="생활패턴-운동")

# 3. ID와 날짜 컬럼을 동적으로 찾기
id_col = next((c for c in alcohol_raw.columns if "비식별키" in c), None)
date_col = next((c for c in alcohol_raw.columns if "날짜" in c), None)

if id_col is None:
    raise KeyError(f"ID 컬럼('비식별키' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")
if date_col is None:
    raise KeyError(f"날짜 컬럼('날짜' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")

# 운동시간(분) 컬럼 찾기
amount_col = next((c for c in alcohol_raw.columns if '운동시간' in c), None)
if amount_col is None:
    raise KeyError(f"운동시간(분) 컬럼을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")

# 4. ID, 날짜, 운동시간 컬럼만 추출
df = alcohol_raw[[id_col, date_col, amount_col]].copy()

# 5. 날짜를 YYYY-MM-DD 문자열로 변환
df['date'] = pd.to_datetime(df[date_col], errors='coerce').dt.strftime("%Y-%m-%d")

# 6. 컬럼명을 통일하여 ['ID', 'date', 'exercise']로 이름 변경
df = df.rename(columns={id_col: 'ID', amount_col: 'exercise'})

# 7. 한 사람(비식별키)이 특정 날짜에 음주 기록이 있으면 alcohol=1, 없으면 해당 행이 존재하지 않음
#    → 즉, 중복 제거만 하면 한 ID/날짜당 한 행이 남고, alcohol 컬럼을 1로 설정
df = df.drop_duplicates(subset=['ID', 'date'], ignore_index=True)

# 8. 최종 컬럼 순서: ['ID', 'date', 'exercise']
df = df[['ID', 'date', 'exercise']]
df = filter_by_valid_ids(df, id_column='ID')

output_path = os.path.join(output_folder, "exercise_per_date.csv")
df.to_csv(output_path, index=False)

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids


# 2. “연구 참여자 기본 정보” 시트를 모두 불러와 합치기
raw = load_raw_file(SYM_raw_paths, sheet_name="연구 참여자 기본 정보")

# 3. 필요한 컬럼을 동적으로 찾기
cols = raw.columns.tolist()

# - ID: “비식별키” 포함
id_col = next((c for c in cols if "비식별" in c), None)

# - Date_of_birth: “Date_of_birth” 또는 “Birth” 등 날짜 포함
dob_col = next((c for c in cols if "Date_of_birth" in c or "생년월일" in c), None)

# - Gender: “Gender” 또는 “성별”
gender_col = next((c for c in cols if "Gender" in c or "성별" in c), None)

ht_col = next(
    (c for c in cols
     if (("height" in c or "키" in c) and "비식별" not in c)),
    None
)

# - Weight: “Weight” 또는 “체중”
wt_col = next((c for c in cols if "Weight" in c or "몸무게" in c), None)

# - Marriage: “Marital” 또는 “혼인” 등
marriage_col = next((c for c in cols if "Marital" in c or "결혼" in c), None)

# - Job: “Occupation” 또는 “직업”
job_col = next((c for c in cols if "Occupation" in c or "직업" in c), None)

# - Smoker history: “Smoker” 또는 “Smoking” 등
smkHx_col = next((c for c in cols if "Smoker" in c or "과거_흡연_여부" in c), None)

# - Drinker history: “Drinker” 또는 “Drinking” 등
drinkHx_col = next((c for c in cols if "Drinker" in c or "음주_여부" in c), None)

# - Suicide history: “Suicide_history” 또는 “자살” 등
suicideHx_col = next(
    (c for c in cols
     if (("Suicide_history" in c or "자살_시도_여부" in c) and "1달" not in c)),
    None
)

# - Suicide ideation in past month: “Suicide_ideation” 또는 “Self_harm” 등
suicide_need_col = next((c for c in cols if "Suicide_ideation" in c or "자살_시도_욕구" in c), None)

# - Medication in past month: “Medication_history” 또는 “Medication” 등
medication_col = next((c for c in cols if "Medication_history" in c or "처방약_여부" in c), None)

# - Consent date: “연구 동의일” 또는 유사 컬럼명
consent_col = next((c for c in cols if "동의" in c and ("연구" in c or "동의일" in c)), None)

# 4. 칼럼 존재 여부 확인

required = {
    "ID": id_col,
    "Date_of_birth": dob_col,
    "Gender": gender_col,
    "Height": ht_col,
    "Weight": wt_col,
    "Marriage": marriage_col,
    "Job": job_col,
    "Smoker_history": smkHx_col,
    "Drinker_history": drinkHx_col,
    "Suicide_history": suicideHx_col,
    "Suicide_ideation": suicide_need_col,
    "Medication_history": medication_col,
    "Consent_date": consent_col,
}

missing = [name for name, col in required.items() if col is None]
if missing:
    raise KeyError(f"다음 필수 컬럼을 찾을 수 없습니다: {missing}\n실제 컬럼명: {cols}")

# 5. 필요한 컬럼만 골라서 복사
demographic_data = raw[
    [
        id_col,
        dob_col,
        gender_col,
        ht_col,
        wt_col,
        marriage_col,
        job_col,
        smkHx_col,
        drinkHx_col,
        suicideHx_col,
        suicide_need_col,
        medication_col,
    ]
].copy()

# 6. '생년월일' 컬럼명을 'Date_of_birth'로 변경
demographic_data.rename(columns={dob_col: "Date_of_birth"}, inplace=True)

# 7. Date_of_birth → datetime64로 변환 (형식: YYYYMMDD)
demographic_data["Date_of_birth"] = pd.to_datetime(
    demographic_data["Date_of_birth"],
    format="%Y%m%d",
    errors="coerce"
)
# 변환 실패한 항목은 NaT가 되며, 이후 계산 시 주의

# Convert consent date column to datetime
if consent_col is None:
    raise KeyError("필수 컬럼 '연구 동의일'을 찾을 수 없습니다. 실제 컬럼명: {}".format(cols))
demographic_data["Consent_date"] = pd.to_datetime(
    raw[consent_col], errors="coerce"
)

# 8. 오늘 날짜 기준으로 나이(age) 계산
# Compute age at consent date
consent_dates = demographic_data["Consent_date"].dt.date
birth_dates = demographic_data["Date_of_birth"].dt.date
years_diff = consent_dates.apply(lambda d: d.year) - demographic_data["Date_of_birth"].dt.year
adjust = ((demographic_data["Consent_date"].dt.month < demographic_data["Date_of_birth"].dt.month)
          | ((demographic_data["Consent_date"].dt.month == demographic_data["Date_of_birth"].dt.month)
             & (demographic_data["Consent_date"].dt.day < demographic_data["Date_of_birth"].dt.day))
         ).astype(int)
ages = (years_diff - adjust).fillna(0).astype(int)

# 9. 'age' 컬럼으로 삽입
demographic_data["age"] = ages

# 10. 이후 더 이상 'Date_of_birth'와 'Consent_date' 컬럼은 필요 없으니 삭제
demographic_data.drop(columns=["Date_of_birth", "Consent_date"], inplace=True)

# 11. 컬럼명 개별 매핑
demographic_data.rename(columns={
    id_col: "ID",
    gender_col: "gender",
    ht_col: "ht",
    wt_col: "wt",
    marriage_col: "marriage",
    job_col: "job",
    smkHx_col: "smkHx",
    drinkHx_col: "drinkHx",
    suicideHx_col: "suicideHx",
    suicide_need_col: "suicide_need_in_month",
    medication_col: "medication_in_month"
}, inplace=True)
# 'age' 컬럼은 이미 올바른 이름이므로 그대로 둡니다.

# 12. 범주형 변수를 1/0으로 변환
demographic_data["marriage"] = demographic_data["marriage"].astype(str).str.upper().eq("Y").astype(int)
demographic_data["job"] = demographic_data["job"].astype(str).str.upper().eq("Y").astype(int)
demographic_data["smkHx"] = demographic_data["smkHx"].astype(str).str.upper().eq("Y").astype(int)
demographic_data["drinkHx"] = demographic_data["drinkHx"].astype(str).str.upper().eq("Y").astype(int)
demographic_data["suicideHx"] = demographic_data["suicideHx"].astype(str).str.upper().eq("Y").astype(int)
demographic_data["suicide_need_in_month"] = (
    demographic_data["suicide_need_in_month"].astype(str).str.upper().eq("Y").astype(int)
)
demographic_data["medication_in_month"] = (
    demographic_data["medication_in_month"].astype(str).str.upper().eq("Y").astype(int)
)
demographic_data["gender"] = demographic_data["gender"].astype(str).str.upper().map({"M": 1, "F": 0}).fillna(0).astype(int)

# 13. 유효한 ID만 필터링
print(demographic_data.columns)
demographic_data = filter_by_valid_ids(demographic_data, id_column="ID")

# 14. 인덱스 리셋 및 저장
demographic_data.reset_index(drop=True, inplace=True)
output_path = os.path.join(output_folder, "demographic_data.csv")
demographic_data.to_csv(output_path, index=False)

Index(['ID', 'gender', 'ht', 'wt', 'marriage', 'job', 'smkHx', 'drinkHx',
       'suicideHx', 'suicide_need_in_month', 'medication_in_month', 'age'],
      dtype='object')


In [13]:
import pandas as pd
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids

# 2. “생활패턴-음주” 시트를 모두 불러와 합치기
alcohol_raw = load_raw_file(SYM_raw_paths, sheet_name="생활패턴-카페인")

# 3. ID와 날짜 컬럼을 동적으로 찾기
id_col = next((c for c in alcohol_raw.columns if "비식별키" in c), None)
date_col = next((c for c in alcohol_raw.columns if "날짜" in c), None)

if id_col is None:
    raise KeyError(f"ID 컬럼('비식별키' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")
if date_col is None:
    raise KeyError(f"날짜 컬럼('날짜' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")

# 섭취량 컬럼 찾기
amount_col = next((c for c in alcohol_raw.columns if '섭취량' in c), None)
if amount_col is None:
    raise KeyError(f"섭취량 컬럼('섭취량' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")

# 4. ID와 날짜만 추출
df = alcohol_raw[[id_col, date_col, amount_col]].copy()

# 5. 날짜를 YYYY-MM-DD 문자열로 변환
df['date'] = pd.to_datetime(df[date_col], errors='coerce').dt.strftime("%Y-%m-%d")

# 6. 컬럼명을 통일하여 ['ID', 'date']로 이름 변경
df = df.rename(columns={id_col: 'ID', amount_col: 'coffee'})

# 7. 한 사람(비식별키)이 특정 날짜에 음주 기록이 있으면 alcohol=1, 없으면 해당 행이 존재하지 않음
#    → 즉, 중복 제거만 하면 한 ID/날짜당 한 행이 남고, alcohol 컬럼을 1로 설정
df = df.drop_duplicates(subset=['ID', 'date'], ignore_index=True)

# 8. 최종 컬럼 순서: ['ID', 'date', 'alcohol']
df = df[['ID', 'date', 'coffee']]
df = filter_by_valid_ids(df, id_column='ID')

output_path = os.path.join(output_folder, "coffee_per_date.csv")
df.to_csv(output_path, index=False)

In [14]:
import pandas as pd
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids


# 2. “생활패턴-음주” 시트를 모두 불러와 합치기
alcohol_raw = load_raw_file(SYM_raw_paths, sheet_name="생활패턴-음주")

# 3. ID와 날짜 컬럼을 동적으로 찾기
id_col = next((c for c in alcohol_raw.columns if "비식별키" in c), None)
date_col = next((c for c in alcohol_raw.columns if "날짜" in c), None)

if id_col is None:
    raise KeyError(f"ID 컬럼('비식별키' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")
if date_col is None:
    raise KeyError(f"날짜 컬럼('날짜' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")

# 음주량 컬럼 찾기
amount_col = next((c for c in alcohol_raw.columns if '음주량' in c), None)
if amount_col is None:
    raise KeyError(f"음주량 컬럼('음주량' 포함)을 찾지 못했습니다. 가능한 컬럼: {alcohol_raw.columns.tolist()}")

# 4. ID와 날짜, 음주량 컬럼만 추출
df = alcohol_raw[[id_col, date_col, amount_col]].copy()

# 5. 날짜를 YYYY-MM-DD 문자열로 변환
df['date'] = pd.to_datetime(df[date_col], errors='coerce').dt.strftime("%Y-%m-%d")

# 6. 컬럼명을 통일하여 ['ID', 'date', 'alcohol']로 이름 변경
df = df.rename(columns={id_col: 'ID', amount_col: 'alcohol'})

# 7. 한 사람(비식별키)이 특정 날짜에 음주 기록이 있으면 alcohol=1, 없으면 해당 행이 존재하지 않음
df = df.drop_duplicates(subset=['ID', 'date'], ignore_index=True)

# 8. 최종 컬럼 순서: ['ID', 'date', 'alcohol']
df = df[['ID', 'date', 'alcohol']]
df = filter_by_valid_ids(df, id_column='ID')

output_path = os.path.join(output_folder, "alcohol_per_date.csv")
df.to_csv(output_path, index=False)

In [15]:
import pandas as pd
import numpy as np
from utils_for_preprocessing import (
    load_raw_file,
    serialize_lifelog_heartrate,
    filter_by_valid_ids
)


sheet_name = "라이프로그-심박수"

# For each path, produce the long-format heart-rate DataFrame
hr_dfs = [serialize_lifelog_heartrate(p) for p in SYM_raw_paths]
HR_melted = pd.concat(hr_dfs, ignore_index=True)
HR_melted.rename(columns={"heart_rate": "HR"}, inplace=True)
HR_melted["HR"] = HR_melted["HR"].fillna(0)
HR_melted["date"] = HR_melted["date"].astype(str).str[:10]
HR_melted = filter_by_valid_ids(HR_melted, id_column='ID')
output_path = os.path.join(output_folder, "HR.csv")
HR_melted.to_csv(output_path, index=False)

print(HR_melted.head(10))

         date         ID      time     HR
0  2022-05-17  SYM1-1-65  00:00:00   79.0
1  2022-05-16  SYM1-1-65  00:00:00    0.0
2  2022-05-15  SYM1-1-65  00:00:00   70.0
3  2022-05-14  SYM1-1-65  00:00:00   72.0
4  2022-05-13  SYM1-1-65  00:00:00  155.0
5  2022-05-12  SYM1-1-65  00:00:00    0.0
6  2022-05-11  SYM1-1-65  00:00:00   77.0
7  2022-05-10  SYM1-1-65  00:00:00   73.0
8  2022-05-09  SYM1-1-65  00:00:00    0.0
9  2022-05-07  SYM1-1-65  00:00:00    0.0


In [16]:
import pandas as pd
import numpy as np
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids
from functools import reduce


steps = load_raw_file(SYM_raw_paths, sheet_name="라이프로그-걸음수")

# ——— 1. SYM1 & SYM2 엑셀병합 & 유효 ID 필터링 ———
# raw_dfs: 각 엑셀파일에서 같은 시트 읽어서 DataFrame 리스트로
# valid_ids 필터링

# ——— 2. 중복 ID·날짜 제거 ———
foot = steps.drop_duplicates(subset=['비식별키', '날짜'], keep='last')


# ——— 4. Mi Band 사용자 데이터 제외 ———
#    '측정_유형' 컬럼이 'Mi Band'인 행을 제거
foot = foot[foot['측정_유형'] != 'MI-BAND']

# ——— 5. 측정값(NaN) 있는 행 제거 ———
foot.dropna(subset=['측정값(_1_:_값_없음)'], inplace=True)



# ——— 8. 불필요 컬럼 제거 ———
foot.drop(['총_걸음수', '측정_유형'], axis=1, inplace=True)

# ——— 9. 인덱스 리셋 ———
foot.reset_index(drop=True, inplace=True)

# ——— 10. 시계열 형태로 변환 (serialize_lifelog_data) ———
#  하루 1440분에 대응하는 시간 인덱스 생성
time_idx = pd.date_range('00:00:00', periods=1440, freq='1min').time
col_minutes = [t.strftime('%H:%M:%S') for t in time_idx]

#  '측정값(-1_:_값_없음)' 컬럼을 쉼표로 분리
split_vals = foot['측정값(_1_:_값_없음)'].str.split(',').tolist()
df_splitted = pd.DataFrame(split_vals, columns=col_minutes)

#  원본과 합치고, 원래 측정 단위·값 컬럼 제거
df_merged = pd.concat([foot[['날짜', '비식별키']].reset_index(drop=True),
                       df_splitted.reset_index(drop=True)], axis=1)

#  melt로 long 포맷 변환
foot_melted = df_merged.melt(id_vars=['날짜', '비식별키'],
                             var_name='time',
                             value_name='foot')

# ——— 11. 컬럼명 정리 & 값 정제 ———
foot_melted.rename(columns={
    '날짜': 'date',
    '비식별키': 'ID'
}, inplace=True)

#  '-1' 값을 np.nan으로 변경
foot_melted.loc[foot_melted['foot'] == '-1', 'foot'] = np.nan


#  date 컬럼을 'YYYY-MM-DD' 포맷 문자열로
foot_melted['date'] = foot_melted['date'].astype(str).str[:10]

foot_melted = filter_by_valid_ids(foot_melted, id_column="ID")


output_path = os.path.join(output_folder, "foot.csv")
foot_melted.to_csv(output_path, index=False)

In [17]:
import pandas as pd
import numpy as np
from utils_for_preprocessing import extract_emotion_diary_from_raw, filter_by_valid_ids
from functools import reduce


# For each metric, extract from both files and concatenate
positive_dfs = [
    extract_emotion_diary_from_raw(
        p,
        questionnaire_sheet='정서일지',
        df_name='positive_feeling',
        questionnaire_column='긍정적기분'
    ) for p in SYM_raw_paths
]
positive = pd.concat(positive_dfs, ignore_index=True)

negative_dfs = [
    extract_emotion_diary_from_raw(
        p,
        questionnaire_sheet='정서일지',
        df_name='negative_feeling',
        questionnaire_column='부정적기분'
    ) for p in SYM_raw_paths
]
negative = pd.concat(negative_dfs, ignore_index=True)

positive_E_dfs = [
    extract_emotion_diary_from_raw(
        p,
        questionnaire_sheet='정서일지',
        df_name='positive_E',
        questionnaire_column='긍정적에너지'
    ) for p in SYM_raw_paths
]
positive_E = pd.concat(positive_E_dfs, ignore_index=True)

negative_E_dfs = [
    extract_emotion_diary_from_raw(
        p,
        questionnaire_sheet='정서일지',
        df_name='negative_E',
        questionnaire_column='부정적에너지'
    ) for p in SYM_raw_paths
]
negative_E = pd.concat(negative_E_dfs, ignore_index=True)

anxiety_dfs = [
    extract_emotion_diary_from_raw(
        p,
        questionnaire_sheet='정서일지',
        df_name='anxiety',
        questionnaire_column='불안'
    ) for p in SYM_raw_paths
]
anxiety = pd.concat(anxiety_dfs, ignore_index=True)

annoying_dfs = [
    extract_emotion_diary_from_raw(
        p,
        questionnaire_sheet='정서일지',
        df_name='annoying',
        questionnaire_column='짜증'
    ) for p in SYM_raw_paths
]
annoying = pd.concat(annoying_dfs, ignore_index=True)

#data preprocessing
data_list = [positive, negative, positive_E, negative_E, anxiety, annoying]
emotion_diary = reduce(lambda x, y : pd.merge(x, y, on=['ID', 'date'], how='outer'), data_list)
emotion_diary = filter_by_valid_ids(emotion_diary, id_column='ID')

output_path = os.path.join(output_folder, "emotion_diary.csv")
emotion_diary.to_csv(output_path, index=False)

In [18]:
import pandas as pd
import numpy as np
from utils_for_preprocessing import load_raw_file, filter_by_valid_ids
from functools import reduce

diary = load_raw_file(SYM_raw_paths, sheet_name="일기")
diary = diary.rename(columns={'비식별키': 'ID', '날짜' : 'date', '기분' : 'mood', '내용' : 'contents'})
diary = filter_by_valid_ids(diary ,id_column="ID")

diary = diary.drop_duplicates(subset=['ID', 'date'], keep='last')
diary.dropna(subset=['contents'], inplace=True)
diary = diary.drop(columns=['시간'])
output_path = os.path.join(output_folder, "diary.csv")
diary.to_csv(output_path, index=False)